# Test processing entities in the csv file

## Data preprocessing

In [2]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('../procesed_data/entities.csv', sep=';')
df = df.dropna()
df['POS'].replace('-', -1, inplace=True)
df['POS'] = df['POS'].astype(int)
df['LAPS'] = df['LAPS'].astype(int)
df['GRID'] = df['GRID'].astype(int)

In [44]:
df.head()

,YEAR,GP NAME,POS,DRIVER NAME,TEAM NAME,LAP TIME,GRID,LAPS,POINTS
0,1973,italian-grand-prix,1,#2 Ronnie Peterson,Team Lotus,1:29:17.0,1,55,9.0
1,1973,italian-grand-prix,2,#1 Emerson Fittipaldi,Team Lotus,+0.8,4,55,6.0
2,1973,italian-grand-prix,3,#8 Peter Revson,McLaren,+28.8,2,55,4.0
3,1973,italian-grand-prix,4,#5 Jackie Stewart,Tyrrell,+33.2,6,55,3.0
4,1973,italian-grand-prix,5,#6 François Cevert,Tyrrell,+46.2,11,55,2.0


In [45]:
df.to_csv('../procesed_data/df_entities.csv', sep=';', index=False)

In [3]:
df = pd.read_csv('../data/procesed_data/df_entities.csv', sep=';')

## Data queries
Ideas what to look for:
- in what races have to drivers met
    - find all rows with specified drivers and return those with matching year and gp name
- look for a specific grand prix and return driver with the most wins at that grand prix
- find a driver with the most ammount of __second__ places when starting from __first__
- find if and when were two pilots collegues

In [4]:
pilot1 = 'leclerc'
pilot2 = 'Lewis Hamilton'
year = 2019

def find_pairs(df, p1, p2):
    filtered_p1 = df[df['DRIVER NAME'].str.contains(p1, case=False)]
    filtered_p2 = df[df['DRIVER NAME'].str.contains(p2, case=False)]
    
    filtered_p1 = filtered_p1[filtered_p1['YEAR'] == year]
    filtered_p2 = filtered_p2[filtered_p2['YEAR'] == year]

    pairs = filtered_p1.merge(filtered_p2, on=['YEAR', 'GP NAME'])
    return pairs[['YEAR', 'GP NAME']].head()


In [5]:
print(f'{pilot1} and {pilot2} raced together in:')
print(find_pairs(df, pilot1, pilot2))

leclerc and Lewis Hamilton raced together in:
   YEAR                   GP NAME
0  2019        russian-grand-prix
1  2019  united-states-grand-prix
2  2019         monaco-grand-prix
3  2019      hungarian-grand-prix
4  2019         french-grand-prix


In [25]:
gp = 'Italian'

def find_entity_wiki(name):
    url = 'https://en.wikipedia.org/wiki/'
    path = url + '_'.join(name.split(' ')[1:]).title()
    page = None
    try:
        with open(f'../data/{path}/page.txt', 'r') as f:
            page = f.read()
    except FileNotFoundError:
        return None
    wins_reges = r'[wW]ins(\d+)'
    wins = re.findall(wins_reges, page)
    if len(wins) == 0:
        return None
    return wins[0]

def find_most_wins(df, gp_name, group_by_tag='DRIVER NAME'):
    filtered = None
    try:
        filtered = df[(df['GP NAME'].str.contains(gp_name, case=False)) & (df['POS'] == 1)]
        filtered = filtered.groupby([group_by_tag]).size().reset_index(name='counts')
        filtered = filtered.sort_values(by=['counts'], ascending=False)
        # filtered.iloc[0, 0]
        # x == row
        # y == column
        filtered['WINS'] = filtered['DRIVER NAME'].apply(find_entity_wiki)
        return filtered.head()
    except KeyError:
        print('Wrong group_by_tag provided')
        return 'No drivers found'

In [26]:
print(find_most_wins(df, gp))

              DRIVER NAME  counts WINS
38     #44 Lewis Hamilton       2  103
3       #1 Max Verstappen       2   49
4   #1 Michael Schumacher       2   91
7     #1 Sebastian Vettel       2   53
33  #3 Michael Schumacher       2   91


In [103]:
pilot1 = 'Bottas'
pilot2 = 'Lewis Hamilton'

def find_collegues(df, p1, p2):
    filtered_p1 = df[df['DRIVER NAME'].str.contains(p1, case=False)]
    filtered_p2 = df[df['DRIVER NAME'].str.contains(p2, case=False)]

    pairs = filtered_p1.merge(filtered_p2, on=['YEAR', 'TEAM NAME'])
    return pairs.drop_duplicates(subset=['YEAR', 'TEAM NAME'])[['YEAR', 'TEAM NAME']].sort_values(by=['YEAR'])

In [104]:
print(find_collegues(df, pilot1, pilot2))

     YEAR TEAM NAME
0    2017  Mercedes
701  2018  Mercedes
289  2019  Mercedes
545  2020  Mercedes
897  2021  Mercedes


In [26]:
df.head()

,YEAR,GP NAME,POS,DRIVER NAME,TEAM NAME,LAP TIME,GRID,LAPS,POINTS
0,1973,italian-grand-prix,1,#2 Ronnie Peterson,Team Lotus,1:29:17.0,1,55,9.0
1,1973,italian-grand-prix,2,#1 Emerson Fittipaldi,Team Lotus,+0.8,4,55,6.0
2,1973,italian-grand-prix,3,#8 Peter Revson,McLaren,+28.8,2,55,4.0
3,1973,italian-grand-prix,4,#5 Jackie Stewart,Tyrrell,+33.2,6,55,3.0
4,1973,italian-grand-prix,5,#6 François Cevert,Tyrrell,+46.2,11,55,2.0


In [38]:
url = 'https://en.wikipedia.org/wiki/'

In [33]:
name = df.iloc[0, 3]

In [39]:
url + '_'.join(name.split(' ')[1:]).title()

'https://en.wikipedia.org/wiki/Ronnie_Peterson'

In [4]:
p1 = 'fernando alonso'
p2 = 'mark webber'
title = '2005 malaysian grand prix'

def find_wins(p1, p2, title):
    results = []
    year = title.split(' ')[0]
    gp_name = '-'.join(title.split(' ')[1:]).lower()
    print(gp_name)

    filtered_p1 = df[(df['DRIVER NAME'].str.contains(p1, case=False)) & (df['YEAR'] == int(year)) & (df['GP NAME']==gp_name)]
    filtered_p2 = df[(df['DRIVER NAME'].str.contains(p2, case=False)) & (df['YEAR'] == int(year)) & (df['GP NAME']==gp_name)]
    pairs = filtered_p1.merge(filtered_p2, on=['YEAR', 'GP NAME'])
    # check if both pilots finished the race
    pos_x = 'DNF' if pairs['POS_x'][0] < 0 else pairs['POS_x'][0]
    pos_y = 'DNF' if pairs['POS_y'][0] < 0 else pairs['POS_y'][0]
    # add print statement to results
    results.append(f'{year} {gp_name}\n{p1.title()} Finished: {pos_x}\n{p2.title()} Finished: {pos_y}')
    _ = [print(r) for r in results]

In [5]:
find_wins(p1, p2, title)

malaysian-grand-prix
2005 malaysian-grand-prix
Fernando Alonso Finished: 1
Mark Webber Finished: DNF


In [37]:
df['POS'][1]

2